In [0]:
from pyspark.sql.functions import *

spark.sql("USE CATALOG real_time_projects")
spark.sql("USE SCHEMA realtime_log_analytics_alerting")

alerts_df = (
    spark.readStream.table("gold_log_metrics")
    .filter(
        (col("level") == "ERROR") &
        ((col("log_count") > 50) | (col("avg_response_time") > 2000))
    )
)

alerts_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/real_time_projects/realtime_log_analytics_alerting/project_volume/checkpoints/alerts") \
    .outputMode("append") \
    .trigger(availableNow=True).table("alerts_table")
